In [5]:
import autograd.numpy as np
from autograd import grad
import workload

In [7]:
n = 4096*2
WtW = workload.AllRange(n).WtW

def loss(B):
    p, n = B.shape
    A = np.vstack([np.eye(n), B])
    A /= A.sum(axis=0)
    AtA1 = np.linalg.pinv(A.T.dot(A))
    return np.trace(AtA1.dot(WtW))

def loss2(B):
    p, n = B.shape
    A = np.vstack([np.eye(n), B])
    A /= A.sum(axis=0)
    AtA1 = np.linalg.inv(A.T.dot(A))
    return np.trace(AtA1.dot(WtW))

def loss3(B):
    p, n = B.shape
    scale = 1.0 + B.sum(axis=0)
    C = WtW * scale * scale[:,None]
    R = np.linalg.inv(np.eye(p) + B.dot(B.T))
    Z = B.T.dot(R.dot(B).dot(C))  
    return np.trace(C) - np.trace(Z)

In [5]:
B = np.random.rand(n//16, n)
% time print loss(B)
% time print loss2(B)
% time print loss3(B)

2.84013172436e+15
CPU times: user 33min 57s, sys: 8min 42s, total: 42min 40s
Wall time: 6min 12s
2.84013172436e+15
CPU times: user 3min 31s, sys: 8.06 s, total: 3min 39s
Wall time: 30.5 s
2.84013172436e+15
CPU times: user 9.5 s, sys: 1.11 s, total: 10.6 s
Wall time: 1.45 s


In [34]:
from scipy import sparse
from scipy.sparse.linalg import lsqr, aslinearoperator

def lstsq(A,y):
    return lsqr(A, y)[0]

def loss(B):
    p, n = B.shape
    A = np.vstack([np.eye(n), B])
    A = A / np.sum(A, axis=0)
    AtA1 = np.linalg.inv(np.dot(A.T, A))
    return np.trace(np.dot(AtA1, WtW))

def loss_and_grad(P, WtW, y):
    """ 
    P: a un-normalized strategy matrix (may be represented as a LinearOperator)
    WtW: a workload matrix (may be represented as a LinearOperator)
    y: a noise vector

    return the objective and gradient in outer product form 
    """
    p, n = P.shape
    P = aslinearoperator(P)

    scale = P.H.dot(np.ones(p))
    D = aslinearoperator(sparse.diags(1.0/scale))

    A = P * D
    At = A.H

    z = lstsq(A, y)
    dz = 2 * WtW.dot(z)
    ans = z.dot(dz) / 2.0

    # term1 = -outer(z, a)
    a = lstsq(At, dz)
    #term2 = outer(b, c)
    b = lstsq(A, a)
    c = y - lstsq(At, At.dot(y))
    # term3 = outer(d, e)
    d = dz - lstsq(A, A.dot(dz))
    e = lstsq(At, z)

    U = np.vstack([-a,c,e])
    V = np.vstack([z,b,d])

    u0 = -np.ones(p)
    v0 = np.sum(V * P.H.dot(U.T).T, axis=0) / scale**2

    return ans, (np.vstack([u0, U]), np.vstack([v0, V/scale]))

def grad2(B, WtW):
    p, n = B.shape
    I = np.eye(n)
    A = np.vstack([np.eye(n), B])
    ans = np.zeros_like(B)
    rep = 1000
    for i in range(rep):
        y = np.random.laplace(loc=0, scale=1.0/np.sqrt(2), size=p+n)
        #y = np.zeros(p+n)
        #y[np.random.randint(p+n)] = 1.0
        obj, (U,V) = loss_and_grad(A, WtW, y)
        ans += U.T[n:].dot(V) / rep
    return ans

n = 16
WtW = workload.AllRange(n).WtW

B = np.random.rand(n//16, n)
print grad(loss)(B)[0]
print grad2(B, WtW)[0]

[  37.30844449  137.27831284  139.14281172    3.87000565   39.83951192
   59.80511987   16.1916421    53.43979421   -9.55225162   24.46103322
   29.30566629   38.94559205   50.22386445   58.48949266   87.08360619
   88.59979094]
[  59.97127711  132.92201275  134.34116411  -16.59664689   45.37346542
   62.43183266   20.14870952   51.00675143  -11.44267155   28.08847923
   38.82191048   37.3492665    61.42166993   81.93514808   82.45440844
   80.34265457]
